In [2]:
from langchain_groq import ChatGroq

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

GROQ_API_KEY =  os.getenv("Groq_API_Key")

llm = ChatGroq(
    temperature=0,
    groq_api_key=GROQ_API_KEY,
    model_name="llama-3.3-70b-versatile"  
)


In [9]:
from langchain_community.document_loaders import WebBaseLoader

loader=WebBaseLoader("https://devjobs.lk/dev-jobs/client/ads/257")
page_data=loader.load().pop().page_content
print(page_data)














DevJobs |             Paid Remote Software Engineer Internship in Sri Lanka | Cadm at Cadmus
                 Apply now for a paid, remote Software Engineer Internship at Cadmus.lk. Join a Sri Lankan digital agency and build real projects using Laravel, React, and APIs.    

























































â˜�ï¸� 094758888400 (OFFLINE)


ğŸ“§ support@devjobs.lk

















ğŸ’¡ Knowledge Base


ğŸ§‘â€�ğŸ’» Developers



Login


Register

























Vishva Vijenayake
R&D Engineer - Synopsys



                       "DevJobs.lk has truly transformed my job search experience. As an R&D Engineer, I constantly look for roles that challenge me and push the boundaries of innovation. DevJobs.lk delivers exactly that with its intuitive interface and carefully curated opportunities. I was impressed by how quickly I connected with companies that value creativity and technical excellence. I would highly recommend DevJobs.lk to any professiona

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [11]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Software Engineer Internship',
  'experience': 'Students or recent graduates',
  'skills': ['HTML',
   'CSS',
   'JavaScript',
   'PHP',
   'Node.js',
   'Laravel',
   'React',
   'RESTful APIs'],
  'description': 'Contribute to real client and internal projects, work with technologies like JavaScript, Laravel, React, Node.js, and APIs, build responsive web apps, backend systems, and custom digital tools'},
 {'role': 'Data Science Internship',
  'experience': 'Not specified',
  'skills': ['Data Science'],
  'description': 'Not specified'}]